In [17]:
import reset
from pathlib import Path
import numpy as np
import tifffile

In [18]:
# Assign variables from reset
animal_loc = reset.animal_loc
animal_id = reset.animal_id
hemi = reset.hemi
frame_len = reset.frame_len

In [19]:
# Define paths for input and output images
image_loc = animal_loc / "ymerged" / hemi
save_loc = animal_loc / "zmerged"

# Ensure save location exists
save_loc.mkdir(parents=True, exist_ok=True)

image_path_list = [f"{image_loc}/{animal_id}_{hemi}_{n}.tif" for n in range(1,3)]

In [22]:
corr_list = []
max_corr_list = []
im1_max_corr_list = []
im2_max_corr_list = []
for fn1 in range (47,frame_len):
    im1 = tifffile.imread(image_path_list[0])
    im1_f = im1[fn1, :, :]
    im2 = tifffile.imread(image_path_list[1])
    for fn2 in range (47,frame_len):
        im2_f = im2[fn2, :, ::-1]
        corr = np.corrcoef(im1_f,im2_f)[0, 1]
        corr_list.append(corr)
    max_corr_list = max(corr_list)
    im2_max_corr = np.argmax(corr_list) + 47
    im2_max_corr_list.append(im2_max_corr+1)
    im1_max_corr_list.append(fn1)
corr_frames = np.argmax(max_corr_list)
im1_corr = im1_max_corr_list[corr_frames]
im2_corr = im2_max_corr_list[corr_frames]
print(f"Image 1: frame {im1_corr}\nImage 2: frame {im2_corr}")

Image 1: frame 47
Image 2: frame 48


In [23]:
image1 = tifffile.imread(image_path_list[0])
im1 = image1[:im1_corr,:,:]
del image1
image2 = tifffile.imread(image_path_list[1])
im2 = image2[:im2_corr,:,:]
del image2
rot2 = im2[::-1,:,::-1]
del im2
img = np.concatenate((im1, rot2), axis=0)
save_path = save_loc / f"{animal_id}_{hemi}.tif"
tifffile.imwrite(save_path, img)
print(f"Saved image: {save_path}")

Saved image: /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/zmerged/rbak006_l.tif
